1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

Метрика hit rate@k говорит о том, были ли в принципе интересны юзеру рекомендации. Precision@k задаёт более строгие условия, поскольку
здесь интересна доля купленных товаров из рекомендованных. Думаю, предпочтительность выбора метрики в первую очередь зависит
от жизненного цикла ML решения. Если проект крайне молодой, например, ещё решается проблема холодного старта, лучше использовать
hit rate, поскольку, скорее всего, с первых разов у нас не получится очень хорошего решения, тут бы глупостей не наделать). 
По мере развития проекта(решении проблемы холодного старта) лучше использовать precision, чтобы с каждой итерацией улучшение
ml метрики приводило к улучшению продукта для юзера.

Но могут быть и такие случаи. Если наши товары довольно высоко эластичны по спросу либо по цене ,например, бытовая техника, ювелирные украшения, то понятно, что, условно,
говоря, в одной сессии юзер скорее всего в принципе не купит несколько видов продукции, поэтому в таких случаях лучше использовать
hit rate@k. Если, наоборот, товары низкоэластичны по спросу или цене, например, бесплатный развлекательный контент, можно сразу использовать
precision, поскольку тут юзер в одной сессии, скорее всего, будет потреблять множество товаров.
А ещё лучше не выводить в рекомендованных товары субситуты, это будет снижать precision@k.

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.

Логарифмирование может сделать функцию гладкой и у нас будет существовать производная слева, справа и в точке. Функция
станет диффиренциируемой и мы можем её оптимизировать, например, мы берём логарифм максимального правдоподобия, когда
обучаемся на антиградиент. Вместо логарифмирования мы можем использовать квадратичную функцию потерь, если хотим
оптимизировать NDCG напрямую. Полиномиальную уже нехорошо, поскольку велик риск переобучения. 
Я думаю, что конкретно в NDCG используется логарифм для того, чтобы была более удобная и правдоподобная шкала измерения. Т.е.
у нас может быть довольно большой перечень рекомендованный товаров(например, лента со скроллом) и поэтому неотлогарифмированный знаменатель
будет тянуть метрику вниз. Наверное поэтому лучше вместо квадратичной функции для NDCG использовать какую нибудь степенную функцию с экспонентой в основании и с обратной дробью в качестве степени, что-то вроде e^1/i, где i-ранг товара.

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

Среднеквадратичная ошибка. Интеракция - время. Было бы актуально для тех сервисов, где в интеракциях довольно много выбросов
и нам нужно их учитывать.

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

In [64]:
import numpy as np

In [65]:
boughted = np.array([1, 3, 5, 7, 9, 11])
recommended = np.array([2, 5, 7, 4, 11, 9, 8, 10, 12, 3])

**Пример выполнения 4го задания**

MSE на  
$y = [0, 2]$  
$y_{pred} = [1, 2]$

MSE = $0.5*((0 - 1)^2 + (2-2)^2) = 0.5$ 

In [4]:
import numpy as np
((np.array([0, 2]) - np.array([1, 2]))**2).mean()

0.5

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)
4/8 = 1/2.

In [66]:
def precision_at_k(boughted, recommended, k=8):
    recommended = recommended[:k]

    precision = np.sum(np.isin(boughted, recommended)) / len(recommended)
    
    return precision

In [67]:
precision_at_k(boughted, recommended, k=8)

0.5

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items) 4/6 = 1/3.

In [68]:
def recall_at_k(boughted, recommended, k=8):
    recommended = recommended[:k]
    
    recall = np.sum(np.isin(boughted, recommended)) / len(boughted)
    
    return recall

In [69]:
recall_at_k(boughted, recommended, k=8)

0.6666666666666666

$$AP@k = \frac{1}{K} \sum{[recommended_{relevant_i}] * precision@k}$$

- K - кол-во рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка рекомендаций

По каждому элементу в recommended считаем precision_at_k, складываем и делим на k.

(1/2 + 2/3 + 4/6 + 3/5)/8 = 0.3041

In [70]:
def ap_k(recommended, boughted, k=8):
    flags = np.isin(recommended, boughted)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k): 
        if flags[i] == True:
            p_k = precision_at_k(recommended, boughted, k=i+1)
            sum_ += p_k
            
    result = sum_ / k
    
    return result

In [71]:
ap_k(recommended, boughted, k=8)

0.35

### NDCG@k
Normalized discounted cumulative gain

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$

Идём по recommended и сравнениваем с bought.

In [ ]:
boughted = np.array([1, 3, 5, 7, 9, 11])
recommended = np.array([2, 5, 7, 4, 11, 9, 8, 10, 12, 3])

- dcg=(0 + 1/log2 + 1/log3 + 0/log4 + 1/log5 + 1/log6 + 0 + 0)/8 = 0.306. 
- ideal_dcg: (1 + 1/log(2) + 1/log(3) + 1/log(4) + 1/log(5) + + 1/log(6) + 1/log(7) + 1/log(8))/8 = 0.58
- ndcg = 0.306/0.58 = 0.52758

In [56]:
def dcg_at_k(recommended, boughted, k=8):
    recommended = recommended[:k]
    
    if len(recommended) == 0:
        return 0.
    
    flags = np.isin(recommended, boughted)
    
    if sum(flags) == 0:
        return 0.
    
    discounts = np.log2(np.arange(2, len(flags)+1))
    discounts = np.insert(discounts, 0, 1)
    
    dsg = np.sum(flags / discounts) / k
    
    return dsg

In [57]:
dcg_at_k(recommended, boughted, k=8)

0.30605738985992403

In [62]:
def ndsg_at_k(recommended, boughted, k=8):
    dcg = dcg_at_k(recommended, boughted, k)
    idcg = dcg_at_k(np.ones(k), np.ones(1), k)
    ndsg = dcg / idcg
    return ndsg

In [63]:
ndsg_at_k(recommended, boughted, k=8)

0.527912744563727

### MRR@k
Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранг первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

1/2 + 1/3 + 1/6 + 1/5 = 0.15

In [82]:
def mean_reciprocal_rank_at_k(recommended, boughted, k=8):
    recommended = recommended[:k]
    ranks=0.
    
    for i, item_rec in enumerate(recommended):
        for item_bought in boughted:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended)

In [83]:
mean_reciprocal_rank_at_k(recommended, boughted, k=8)

0.15